<h1>Go scoring function

The code below is about evaluating model according to their actual X-ray data. Starts with importing necessary modules.

In [ ]:
from __future__ import division
import sys
import math

First function that defined below takes first imput as native model(sys.argv[1]).

In [ ]:
def go_score1(filename1):

    file1 = open(filename1) #native

Goes through pdb file and stores x,y,z coordinates to the created lists.

In [ ]:
    x1=[]
    y1=[]
    z1=[]
    for line in file1:
        if 'ATOM' in line and 'CA' in line:
            splitted_line = [line[:6], line[6:11], line[12:16], line[17:20], line[21], line[22:26], line[30:38], line[38:46], line[46:54]]
            a = "%-6s %5s %4s %3s %s %4s    %8s %8s %8s\n"%tuple(splitted_line)
            a = a.split()
            x1.append(float(a[-3]))
            y1.append(float(a[-2]))
            z1.append(float(a[-1]))

Than it calculates distance within every pair of atom that given pdb structure has and returns these distances as list again

In [ ]:
    distance_native = []
    for i in range(len(x1)-1):
        for j in range(i+1,len(x1)):
            val = math.sqrt((x1[i]-x1[j])**2+(y1[i]-y1[j])**2+(z1[i]-z1[j])**2)
            distance_native.append(val)
            
    file1.close()

    return distance_native


Second function does same thing for second imput(sys.argv[2])

In [ ]:
def go_score2(filename2):

    file2 = open(filename2) #models
    x2=[]
    y2=[]
    z2=[]
    for line in file2:
        if 'ATOM' in line and 'CA' in line:
            splitted_line = [line[:6], line[6:11], line[12:16], line[17:20], line[21], line[22:26], line[30:38], line[38:46], line[46:54]]
            a = "%-6s %5s %4s %3s %s %4s    %8s %8s %8s\n"%tuple(splitted_line)
            a = a.split()
            x2.append(float(a[-3]))
            y2.append(float(a[-2]))
            z2.append(float(a[-1]))
    distance_model = []
    for i in range(len(x2)-1):
        for j in range(i+1,len(x2)):
            val = math.sqrt((x2[i]-x2[j])**2+(y2[i]-y2[j])**2+(z2[i]-z2[j])**2)
            distance_model.append(val)
            
    file2.close()

    return distance_model


In this part main function runs.

$$\frac{\sum_{k=1}^n e^{((-(r_n - r_m))^2)/{2*\sigma^2}}}{n}$$ 
This summation goes through distance between every pair of atoms and subtract the distance in the model from the distance in the x-ray structure for same atom pair. Summation is divided by the number of atom pairs for normalization.

In [ ]:
if __name__ == "__main__":
    dist_nat = go_score1(sys.argv[1])
    dist_mod = go_score2(sys.argv[2])
    
    val = 0
    sigma = 1
    for i in range(len(dist_nat)):
        calc = math.exp((-(dist_nat[i]-dist_mod[i])**2)/2*sigma**2)
        val = val + calc
    result = val / len(dist_nat)

    print result


Best possible score that we can get by using this function is 1, which we can see that below by using same x-ray structure as model of itself.

In [4]:
ls

2mta.B99990004.pdb  2mta_cut.pdb  go_score.py  ip_test.ipynb


In [8]:
%run go_score.py 2mta_cut.pdb 2mta_cut.pdb

1.0


In [13]:
%run go_score.py 2mta_cut.pdb 2mta.B99990004.pdb

0.26062108512


In [15]:
%run go_score.py 2mta.B99990004.pdb 2mta.B99990004.pdb

1.0


In [16]:
%run go_score.py 2mta.B99990004.pdb 2mta_cut.pdb

0.26062108512


Modifying last part of script as below makes it easier to evaluate non-variable outputs which was modellers outputs in this example.

In [ ]:
if __name__ == "__main__":
    dist_nat = go_score1(sys.argv[1])

    for name in range(10):
        name = '%s.B%i.pdb'%(sys.argv[2],int(99990001)+int(name))

        dist_mod = go_score2(name)
    
        val = 0
        sigma = 1
        for i in range(len(dist_nat)):
            calc = math.exp((-(dist_nat[i]-dist_mod[i])**2)/2*sigma**2)
            val = val + calc
        result = val / len(dist_nat)

        print name,' ',result
